In [ ]:
# import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns

from scipy.stats import norm, zscore

#turn off warnings
import warnings
warnings.filterwarnings("ignore")

## Part 1. Data Cleaning ##

In [ ]:
# loading the data

df = pd.read_excel('Census-mod.xlsx')

df.head(n=10)

In [ ]:
# Examining data types of each column
df.dtypes

In [ ]:
# Show how many rows and columns this dataset has

print("Number of rows : ", df.shape[0])
 
# Obtaining the number of columns
print("Number of columns : ", df.shape[1])

### 1.1. Convert Columns to Correct Data Type ###

In [ ]:
# Convert the categorical columns to a categorical datatype without numeric encoding
cat_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
for column in cat_columns:
    df[column] = df[column].astype('category')

# Verify the change by displaying the datatypes again
df.dtypes

### 1.2. Removal of Duplicates ###

In [ ]:
# Tabulate and identify if there any duplicate records (not duplicate cells) in which case you need to remove them. 
# Identify these duplicates in your report. 

pd.DataFrame(df)
duplicate_records = df[df.duplicated(keep=False)]

if not duplicate_records.empty:
    print("\nDuplicate Records:")
else:
    print("\nNo Duplicate Records Found.")

display(duplicate_records)

In [ ]:
# Remove duplicated records

census = df.drop_duplicates(keep='first')

print("\nDataFrame after removing duplicates:")
display(census)

### 1.3. Fix Spelling Errors ###

In [ ]:
# identifying unique values across categorical variables
cat_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']

unique_values_list = []

# Iterate through columns of interest and populate the list with unique values and column names
for column in cat_columns:
    unique_values = sorted(census[column].unique())
    for value in unique_values:
        unique_values_list.append([column, value])

# Create a DataFrame from the list with appropriate column names
unique_values_df = pd.DataFrame(unique_values_list, columns=['Column_Name', 'Unique_Value'])

# Print the resulting DataFrame
pd.set_option('display.max_rows', None)
print("DataFrame with column names and unique values:")
display(unique_values_df)

In [ ]:
# Consistency of spelling within categorical variables

# marital-status
census.loc[:, 'marital-status'] = census['marital-status'].str.replace('married-AF-spouse', 'Married-AF-spouse')

# relationship
census.loc[:, 'relationship'] = census['relationship'].str.replace('husband', 'Husband')
census.loc[:, 'relationship'] = census['relationship'].str.replace('wife', 'Wife')

# sex
census.loc[:, 'sex'] = census['sex'].str.replace('Mole', 'Male')
census.loc[:, 'sex'] = census['sex'].str.replace('Femole', 'Female')

# native-country 
census.loc[:, 'native-country'] = census['native-country'].str.replace('Hong', 'Hong-Kong')
census.loc[:, 'native-country'] = census['native-country'].str.replace('South', 'South-Korea')
census.loc[:, 'native-country'] = census['native-country'].str.replace('Holand-Netherlands', 'Holland-Netherlands')

### 1.4. Imputing Missing Data ###

Categorical variables:

In [ ]:
# Missing values "?" within categorical variables

for column in cat_columns:
    census.loc[:, column] = census[column].str.replace('?', 'Others')

In [ ]:
## Check data ##
cleaned_unique = []

# Iterate through columns of interest and populate the list with unique values and column names
for column in cat_columns:
    unique_values = sorted(census[column].unique())
    for value in unique_values:
        cleaned_unique.append([column, value])

# Create a DataFrame from the list with appropriate column names
unique_df = pd.DataFrame(cleaned_unique, columns=['Column_Name', 'Unique_Value'])

# Print the resulting DataFrame
pd.set_option('display.max_rows', None)
print("DataFrame with column names and unique values:")
display(unique_df)

Numerical variables:

In [ ]:
# Check for missing values in numerical columns 
num_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

missing_values = []

# Loop through numerical columns and check for missing values (represented by 99999)
for column in num_columns:
    if (census[column] == 99999).any():
        missing_values.append(column)

missing_values

In [ ]:
# Inspect missing values in capital-gain
missing = census['capital-gain'] == 99999
missing_indices = census[missing].index

# Original
census[missing]

# Suggested (to save space)
print(f"Number of rows with missing capital gain: {len(census[missing])}")

In [ ]:
# Missing values within capital-gain have income >50k, examine distribution of >50k and <= 50K

income_groups = ['>50K', '<=50K']

# Create subplots to visualize the normal distribution of 'capital-gain' for each income group
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

for i, group in enumerate(income_groups):
    # Filter data for the current income group
    group_data = census[census['income'] == group]['capital-gain']
    
    # Calculate mean and standard deviation of 'capital-gain' for the current group
    mu, std = group_data.mean(), group_data.std()
    
    # Generate values for the x-axis
    x = np.linspace(mu - 3*std, mu + 3*std, 100)
    
    # Calculate the PDF (Probability Density Function) for the normal distribution
    pdf = norm.pdf(x, mu, std)
    
    # Plot the normal distribution curve
    axes[i].plot(x, pdf, 'r-', label='Normal Distribution')
    axes[i].hist(group_data, bins=10, density=True, alpha=0.7, label='Data Distribution')
    axes[i].set_title(f'Normal Distribution of Capital Gain for {group} Income')
    axes[i].set_xlabel('Capital Gain')
    axes[i].set_ylabel('Density')
    axes[i].legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the mean of 'capital-gain' for the income group '>50K' excluding values of 99999
mean_capgain = census[(census['income'] == '>50K') & (census['capital-gain'] != 99999)]['capital-gain'].mean()

# Replace missing values (99999) in 'capital-gain' for income group '>50K' with the calculated mean
census.loc[(census['income'] == '>50K') & (census['capital-gain'] == 99999), 'capital-gain'] = mean_capgain

(census['capital-gain'] == 99999).any()

In [ ]:
income_groups = ['>50K', '<=50K']

# Create subplots to visualize the normal distribution of 'capital-gain' for each income group
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

for i, group in enumerate(income_groups):
    # Filter data for the current income group
    group_data = census[census['income'] == group]['capital-gain']
    
    # Calculate mean and standard deviation of 'capital-gain' for the current group
    mu, std = group_data.mean(), group_data.std()
    
    # Generate values for the x-axis
    x = np.linspace(mu - 3*std, mu + 3*std, 100)
    
    # Calculate the PDF (Probability Density Function) for the normal distribution
    pdf = norm.pdf(x, mu, std)
    
    # Plot the normal distribution curve
    axes[i].plot(x, pdf, 'r-', label='Normal Distribution')
    axes[i].hist(group_data, bins=10, density=True, alpha=0.7, label='Data Distribution')
    axes[i].set_title(f'Normal Distribution of Capital Gain for {group} Income')
    axes[i].set_xlabel('Capital Gain')
    axes[i].set_ylabel('Density')
    axes[i].legend()

# Show the plots
plt.tight_layout()
plt.show()

### 2. Correlation between Variables ###

In [ ]:
# Plot scatter matrix
pd.plotting.scatter_matrix(census, alpha=0.5, figsize=(20, 12), diagonal='hist')
# Turn off x and y axis ticks for all subplots for greater clarity
for ax in plt.gcf().get_axes():
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)

plt.show()

In [ ]:
#calculate correlation
correlation_matrix = census.corr()
correlation_matrix

Observations from the scatter matrix:  
* Most of the scatter plots do not show a clear linear relationship, suggesting that these variables are relatively independent of each other.  
* Variables like 'capital-gain' and 'capital-loss' have many 0 values, which makes sense since not everyone has capital transactions.  

Observations from the correlation matrix:  
* None of the variables have a correlation coefficient close to 1 or -1 (the highest absolute correlation coefficient is 0.147), which suggests that there is little collinearity between any two variables.
* 'age' has a slight positive correlation with 'capital-gain' and 'hours-per-week'. This means that as an individual's age increases, their capital gain and hours worked per week tend to increase slightly.  
* 'education-num' has a positive correlation with 'capital-gain' and 'hours-per-week'. This means that individuals with higher education levels may have higher capital gains and work more hours per week.  
* 'capital-gain' and 'capital-loss' have a slight negative correlation, so when one increases, the other tends to decrease.  

**Are all variables necessary for analysis?**  
Yes. As multicollinearity is not observed, all variables provide insights on and influence the target variable 'income'.

### 3. Outlier Detection

We detected outliers using seaborn's boxplot; for each variable, data points which lie above the third quartile plus 1.5 times the inter-quartile range are determined to be outliers. To better visualize the data, we normalized them using z-score. 

#### observations

From the diagram below, we observed that the outliers for capital-gain and capital-loss above the 3rd quartile, and exhibit high degrees of variability. On average, the proportion of outliers for both features is approximately 4.7%. Further analysis of the distributions revealed that these two metrics are extremely left-skewed, with most people having capital gains and losses of 0. Taken together, this suggests that most people do not invest, and that the performance of those who do vary significantly.

We also observed that the hours-per-week variable has outliers beyond both the 1st and 3rd quartiles, with a moderate level of variability - in total, they make up 7.9% of the data points. From the histogram, we also observed that the mode lies at approximately 40 hours per week and that the distribution is relatively light-tailed on both extremes. This means that while there are individuals who work significantly shorter or longer hours, working hours of the population as a whole remain rather consistent.

Lastly, there are also a large number of outliers for the age variable, but the variability is relatively smaller; the dataset consists of 8.31% outliers for age. The mean age within the population is 38.6, with a standard deviation of 13.7. The histogram displays a left-skewed distribution, indicating that there are a substantial number of young people within the population. This is a positive sign, as a left-skewed age distribution often suggests a higher proportion of individuals in their prime working years, contributing to economic productivity and vitality.

In [ ]:
# Show outliers using boxplot (only for numeric variables since boxplots can only capture outliers based on standard deviation)
plt.figure(figsize=(20, 6))

# Normalize data using zscore
sns.boxplot(data=zscore(census[[i for i in num_columns if i != "education-num"]]))

In [ ]:
# Generate a column for each numeric variable indicating if a row is an outlier for the given variable
# Based on |zscore| threshold of 1.5, in line with boxplot thresholds

data = []
filtered_num_cols = [i for i in num_columns if i not in ['education-num', 'fnlwgt']]

for c in filtered_num_cols:
    zs = zscore(census[c])
    df[f"is_outlier_{c}"] =  zs > 1.5
    outlier_prop = len(zs[zs>1.5])/len(census)*100
    data.append(outlier_prop)

plt.figure(figsize = (10, 5))
plt.bar(filtered_num_cols, data)
for i, v in enumerate(data):
    plt.text(i, v+0.1, f"{round(v, 2)}%", ha='center')
plt.xlabel('Numeric Variables')
plt.ylabel('Percentage of Outliers')
plt.title('Percentage of Outliers in Numeric Variables')
plt.show()

As the threshold of |z-score| > 1.5 is relatively conservative, a decent number of data points have been identified as outliers.  

Observations:  
* The percentage of outliers for variables like 'age', 'education-num', and 'hours-per-week' is relatively high. However, it does not make sense to ignore these variables based solely on their outlier percentages, as they provide valuable insights into the target variable 'income'.  
* Variables like capital-gain and capital-loss inherently have a high concentration of values at 0, meaning many individuals did not have any capital transactions. The outliers in these variables might be significant events or transactions that could be of interest.  
* Some outliers may be realistic when we consider the context. For instance, while working 80 hours a week is statistically an outlier, it might be a valid data point in specific industries or professions.  

**Should any of the variables be ignored?**  
No, as the outliers percentage is high due to the nature of the variables as explained above. Additionally, the variables all provide insight to the target variable, 'income'.

### 4. Frequency/Density Distribution for Each Feature ###

In [ ]:
from scipy.stats import skew, kurtosis

# Initialize a dictionary to store the computed statistics for each variable
stats_dict = {}

# Iterate over numerical columns to compute the statistics and generate the plots
for col in [i for i in num_columns if i not in ['education-num', 'fnlwgt']]:
    # Plotting the frequency/density distribution
    plt.figure(figsize=(5, 3))
    sns.histplot(census[col], stat='density', kde=True, bins=30)
    plt.title(f"Frequency Density Distribution of {col}")
    plt.ylabel('Density')
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()
    
    # Calculate statistics
    stats = {}
    stats['Mode'] = census[col].mode()[0]
    stats['Median'] = census[col].median()
    stats['Mean'] = census[col].mean()
    stats['Variance'] = census[col].var()
    stats['Std. Dev.'] = census[col].std()
    stats['Skewness'] = skew(census[col])
    stats['Kurtosis'] = kurtosis(census[col])
    
    stats_dict[col] = stats
    
    # Print statistics under each graph
    print("--- Central Tendency ---")
    for key,value in list(stats.items())[:3]:
        print(key, ':', value)
        
    print("--- Variation ---")
    for key,value in list(stats.items())[3:5]:
        print(key, ':', value)
    
    print("--- Shape ---")
    for key,value in list(stats.items())[5:]:
        print(key, ':', value)    

# Print stats in a dataframe for easy comparison
stats_df = pd.DataFrame(stats_dict).T
stats_df

**Observations/Comments:**  

age  
* The graph shows the distribution of the number of individuals of a certain age.  
* There is a peak around 20-40 years.  
* The mean age is approximately 38.65 years.  
* A positive skewness of 0.557 suggests a right-skewed distribution.  * A negative kurtosis value of -0.187 suggests a light tail with  less extreme large jumps away from the mean than a normal distribution.  

education-num:  
* The graph shows the distribution of the number of individuals of a certain education level.  
* There is a peak around 9 and 13, corresponding to high school and bachelor's degrees respectively.  
* The mean education level is around 10.  
* The skewness value of -0.314 is above -0.5, which means that the distribution is fairly symmetrical.  
* A positive kurtosis value of 0.621 suggests heavy tails with more frequent large jumps away from the mean than a normal distribution.  

capital-gain:  
* The graph shows the distribution of the number of individuals of a certain capital gain.  
* There is a peak at 0, which suggests that the majority of individuals have no capital gains.  
* The mean capital gain is around 590, but the high standard deviation indicates a wide spread.  
* The high positive skewness value of 5.91 suggests a very right-skewed distribution.  
* The high positive kurtosis value of 43.5 suggests heavy tails.  

capital-loss:  
* The graph shows the distribution of the number of individuals of a certain capital loss.  
* There is a peak at 0, which suggests that the majority of individuals have no capital losses.  
* The mean capital loss is around 87.6.  
* The high positive skewness value of 5.47 suggests a very right-skewed distribution.  
* The high positive kurtosis value of 20.0 suggests heavy tails.   

hours-per-week:  
* The graph shows the distribution of the number of individuals of a certain number of working hours per week.  
* There is a significant peak at 40 hours, which aligns with the standard workweek.  
* The mean is around 40.43 hours, closely matching the mode.  
* The skewness value of 0.240 is below 0.5, which means that the distribution is fairly symmetrical.  
* The positive kurtosis value of 2.95 suggests heavy tails, with some individuals working significantly more than 40 hours.

### 5. Frequency Distribution for Each Feature by Label Classes ###

In [ ]:
#separate dataframes based on y variable
gt_50 = census.loc[census["income"] == ">50K",:]
lte_50 = census.loc[census["income"] == "<=50K",:]

fig, ax = plt.subplots(13, 2, figsize = (15, 50))
all_cols = [c for c in [*num_columns, *cat_columns] if c != "income"]

for i in range(0, len(all_cols)):
    #calculate col and row number for subplot
    col_name = all_cols[i-1]

    #plot into the designated cell
    ax[i, 0].hist(census[col_name], color = "orange")
    ax[i, 0].hist(gt_50[col_name], color = "green", alpha = 0.7, label = ">50K")
    ax[i, 0].set_title(f"Variable: {col_name} (Income > $50K)")

    ax[i, 1].hist(census[col_name], color = "orange")
    ax[i, 1].hist(lte_50[col_name], color = "red", alpha = 0.7, label = "<=50K")
    ax[i, 1].set_title(f"Variable: {col_name} (Income <= $50K)")

    ax[i, 0].set_ylabel("Frequency")
    ax[i, 1].set_ylabel("Frequency")
    
    #show legend based on labels
    ax[i, 0].legend()
    ax[i, 1].legend()

    #rotate x tick label if it's a categorical variable
    if col_name in cat_columns:
        for tick in ax[i, 0].get_xticklabels():
            tick.set_rotation(90)
        for tick in ax[i, 1].get_xticklabels():
            tick.set_rotation(90)

#set spacing
plt.subplots_adjust(left=-0.2, bottom=0, right=0.85, top=1.5, wspace=0.1, hspace=1.3)

plt.show()

In [ ]:
selected_cols = ['age', 'hours-per-week', 'education-num']


fig, ax = plt.subplots(1, len(selected_cols), figsize = (16, 8))

for i, col_name in enumerate(selected_cols):
    sns.boxplot(data=census, x='income', y=col_name, ax=ax[i], palette={"<=50K": "blue", ">50K": "orange"})    
    ax[i].set_title(f"Plot of {col_name} for Different Income Groups")
    ax[i].set_xlabel("Income")
    ax[i].set_ylabel(col_name)

plt.tight_layout()
plt.show()


**Frequency Distribution Observations:**  
The frequency distribution graphs show the distribution of the stated variable in blue, grouped by income >50K on the left and <=50K on the right.  

**<font color='red'>please help check and supplement the observations cos idk what more to say</font>**  

native-country:  
<font color='red'>idk what to say</font>  

age:  
* People earning more than 50K generally tend to be older, with the median age being higher than that of the <=50K group. This might indicate experience and seniority.  

education-num and education:  
* Higher education levels (like Bachelors, Masters, and Doctorate) have a relatively higher representation in the >50K group.  
* HS-grad and some college education dominate the <=50K group.  

capital-gain and capital-loss:  
* Most individuals with a low capital gain/loss or a capital gain/loss of 0 have income <=50K.  
* This suggests that individuals with higher income have more means to invest in capital and make capital gains/losses.  

hours-per-week:  
* Those earning more than 50K tend to work longer hours, with a notable number of individuals working more than 40 hours a week.  
* For those earning 50K or less, the majority work around 40 hours, but there is also a significant portion working less than that.  

work-class:  
* For both individuals earning >50K and <=50K, most of them fall under the private work class.  

marital-status:  
* For individuals earning >50K, most of them fall under the Married-civ-spouse status.  
* For individuals earning <=50K, most of them fall under the Never-married and Married-civ-spouse statuses.  

occupation:  
* For individuals earning >50K, most of them fall under exec-managerial roles.  
* For individuals earning <=50K, most of them fall under exec-managerial, others and sales roles.  

relationship:  
* For individuals earning >50K, most of them fall under Husband roles.  
* For individuals earning <=50K, most of them fall under Not-in-family roles.  

race:  
* The majority of individuals in both income groups are White. However, when comparing the proportions, Whites and Asians have a higher representation in the >50K group relative to their total population than other races.  

sex:  
* A significant number of males earn more than 50K compared to females. Conversely, more females earn 50K or less.  


**Box Plot Observations:**  

* For age, hours-per-week and education-num, the interquartile range is wider for the >50K group, indicating more variability. Outliers are evident, especially in the hours-per-week distribution for both income groups, suggesting some individuals work exceptionally long or short hours.  
* In summary, variables like age, education, and hours-per-week show distinct patterns between the two income groups, which could be vital predictors in modeling income. The differences in gender and race distributions suggest societal or sector-specific income disparities that might be worth exploring further.

## 6. Predictive Models

We then exported the data for further transformation and storyboarding on SAC. After which, we re-imported the data back to python for to generate predictive models. In one-hot encoding the categorical variables, we prioritised the dropping of the 'Others' class. This is relevant to our feature selection process, which we'll discuss further in the next step.

In [ ]:
##import transformed data, remove fnlwgt, education-num, marital-status, occupation, capital-gain, capital-loss, income
#these categorical variables are redundant because there are other columns which conveys the same information as them or have already been aggregated into other columns. 

#for instance, education indicates the highest education level attained and education-num indicates the number of years spent studying. these two will be correlated
#since the longer you spend studying (typically), then the higher your education level should be.


#function to one-hot encode a dataframe of categorical variables
def get_dummies_from_df(df):
    dummies = pd.DataFrame()
    for c in df.columns:
        #check if categorical variable has 'Others' class; if yes, drop it  when one-hot encoding
        if 'Others' in df[c].unique():
            d = pd.get_dummies(df[c], prefix = c)
            d = d.drop(f"{c}_Others", axis = 1)
        #else, drop the first class when one-hot encoding
        else:
            d = pd.get_dummies(df[c], drop_first = True, prefix = c)

        #concatenate dummies column-wise
        if dummies.empty:
            dummies = d
        else:
            dummies = pd.concat([dummies, d], axis = 1)
        
    #return dummies
    return dummies

redundant_cols = ["age", "workclass", "education", "education-num", "marital-status", "relationship", "occupation", "capital-gain", "capital-loss", "hours-per-week", "income", "fnlwgt"]

df2 = pd.read_csv("Census-mod-cleaned-excel-values.csv").drop(redundant_cols, axis = 1)
df2.head()

In [ ]:
from sklearn.model_selection import train_test_split

#split into training and testing, one hot encode X
X = df2.drop("Income-Grp", axis = 1)
X = get_dummies_from_df(X)
y = df2["Income-Grp"]

#90% to training, 10% to testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve

#function to get class which was dropped during one-hot encoding
def get_dropped_class(original_col, original_df, new_df):
    original_classes = original_df[original_col].unique()
    print(original_classes)
    one_hot_encoded_classes = [col.replace(original_col+"_", "") for col in new_df.columns if original_col in col]
    print(one_hot_encoded_classes)
    dropped = [c for c in original_classes if c not in one_hot_encoded_classes]
    return dropped

#function to turn insignificant features into key-value pairs of categorical variable and classes
def generate_insignificant_dict(insignificant_features):
    dict = {}
    cat_vars = set([cls.split("_")[0] for cls in insignificant_features])
    
    for var in cat_vars:
        cat_classes = [i.split(var+"_")[1] for i in insignificant_features if var in i]
        dict[var] = cat_classes

    return dict

#function to evaluate model performance
def evaluate_model(model):
    
    #predict and evaluate
    y_pred = model.predict(X_test)
    y_pred_tr = model.predict(X_train)

    # Predict probabilities
    y_pred_proba_train = model.predict_proba(X_train)[:, 1] # Use the probabilities for the positive class (class = 1)
    y_pred_proba_test = model.predict_proba(X_test)[:, 1]

    accuracy = model.score(X_test, y_test)
    accuracy_tr = model.score(X_train, y_train)

    # AUC ROC using probabilities
    auc_train = roc_auc_score(y_train, y_pred_proba_train)
    auc_test = roc_auc_score(y_test, y_pred_proba_test)


    #weighted f1 since positive cases only make up 23.9% of the dataset; significant data imbalance
    weighted_f1 = f1_score(y_test, y_pred, average = "weighted")
    weighted_f1_tr = f1_score(y_train, y_pred_tr, average = "weighted")
    #regular f1 score
    f1 = f1_score(y_test, y_pred)
    f1_tr = f1_score(y_train, y_pred_tr)

    #confusion_matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_matrix = pd.DataFrame(conf_matrix, index = [0, 1], columns = [0, 1])

    #other metrics from cf
    cf_vals = conf_matrix.values
    tpr = cf_vals[1,1]/(cf_vals[1,0] + cf_vals[1,1])
    fpr = cf_vals[0,1]/(cf_vals[0, 1] + cf_vals[0, 0])
    tnr = cf_vals[0,0]/(cf_vals[0,0] + cf_vals[0,1])
    fnr = cf_vals[0, 1]/(cf_vals[0, 1] + cf_vals[1, 1])
    precision = cf_vals[1, 1]/(cf_vals[1, 1] + cf_vals[0, 1])

    #output
    print("="*80)
    print(f"training accuracy: {accuracy_tr:.4f} | testing accuracy: {accuracy:.4f}")
    print(f"training f1: {f1_tr:.4f} | training weighted f1 {weighted_f1_tr:.4f}")
    print(f"testing f1: {f1:.4f} | testing weighted f1 {weighted_f1:.4f}")
    print(f"training auc: {auc_train:.4f} | testing auc: {auc_test:.4f}\n\n")
    print("="*80)
    print(f"TPR (Recall or Sensitivity): {tpr:.4f}, TNR (Specificity): {tnr:.4f}")
    print(f"Precision: {precision:.4f}, FPR: {fpr:.4f}, FNR: {fnr:.4f}\n\n")
    print("="*80)
    print("confusion matrix:")
    print(conf_matrix)


    #plot auc roc
    fpr_tr, tpr_tr, threshold_tr = roc_curve(y_train, y_pred_proba_train, pos_label=1)
    fpr_te, tpr_te,  threshold_te = roc_curve(y_test, y_pred_proba_test, pos_label=1)

    plt.figure(figsize = (10, 5))
    plt.plot([0, 1], [0, 1], color = 'navy', linestyle = '--')
    plt.plot(fpr_tr, tpr_tr, label = "ROC (training)")
    plt.plot(fpr_te, tpr_te, label = "ROC (testing)")
    plt.ylabel("TPR")
    plt.xlabel("FPR")
    plt.legend()
    plt.title(f"AUC for {type(model)}")

    basic_metrics  = {
        "accuracy_train": accuracy_tr,
        "auc_train": auc_train,
        "f1_train": f1_tr,
        "accuracy_test": accuracy,
        "auc_test": auc_test,
        "f1_test": f1
    }

    cf_metrics = {
        "tpr": tpr,
        "fpr": fpr,
        "tnr": tnr,
        "fnr": fnr,
        "precision": precision
    }

    return {
        "cf_metrics": cf_metrics,
        "basic_metrics": basic_metrics
    }

### 6.1. Feature Selection

We first formed a logistic regression model with L1 regularization. The purpose of doing this is to make use of Lasso Regression's ability to identify insignificant features by means of assigning coefficients of 0 to them. From the model output, we can see that there are several redundant one-hot encoded features from the original workclass-grp, native-country and education-grp categorical variables. 

With these information, we then simplified these columns by grouping these insignificant classes together with the 'Others' class if it exists as a class within the original categorical columns - otherwise, we group them together and create a new class called 'Others'. This reduced the number of one-hot encoded features from 75 to 58, which in turn led to a significant reduction in the complexity of the models we'll create later.

In [ ]:
#l1 regularization
model_lr_l1 = LogisticRegression(penalty = "l1", solver = "liblinear", random_state = 42).fit(X_train, y_train)

#get insignificant features
insignificant_features = X_train.columns[model_lr_l1.coef_[0] == 0]

#generate dictionary of insignificant classes for each of the original categorical variables
insignificant_classes = generate_insignificant_dict(insignificant_features)

#output
insignificant_classes

In [ ]:
#deep copy of df2 as df3 
df3 = df2.copy(deep = True)

#simplification of the original categorical variables by grouping insignificant classes together under the class 'Others'
for k, v in insignificant_classes.items():
    df3.loc[df3[k].isin(v), k] = 'Others'

#redo train-test-split with df3 (the simplified df with insignificant classes dropped)
X = df3.drop("Income-Grp", axis = 1)
X = get_dummies_from_df(X)
y = df3["Income-Grp"]

#90% to training, 10% to testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

#save training and testing datasets for use on SAC (to be consistent)
#pd.concat([y_train, X_train], axis=1).to_csv("training_set_new.csv", index=False)
#pd.concat([y_test, X_test], axis=1).to_csv("testing_set_new.csv", index=False)

### 6.2. Logistic Regression

The logistic regression model has high accuracy rate both in and out of sample. Its training and testing F1 scores are also quite good, at 0.6426 and 0.6327 respectively. From the ROC curve, we can see that the area under the curve is also very large, indicating that the model can effectively distinguish between data points from the two income groups. Additionally, the difference in in-sample and out-of-sample AUC is also very minimal, which means that the model performs very consistently and generalizes well to unseen data.

From the confusion matrix, we can see that the model has a moderate recall of 0.5457. This means that the model managed to correctly identify approximately half of the positive data points. However, the low FPR of 0.0566 means that its positive predictions are very reliable. Conversely, while the model managed to correctly identify most of the negative data points, the false negative rate of 0.2473 means that its negative predictions are far less reliable than its positive ones.

In [ ]:
#fit logistic
model_lr = LogisticRegression().fit(X_train, y_train)
summary_lr = evaluate_model(model_lr)

### 6.3. Decision Tree Classifier

In our decision tree classifier, the training accuracy of 0.8823 is significantly higher than the testing accuracy of 0.8321. This discrepancy can also be observed in the training and testing f1 scores of 0.7265 and 0.6007 respectively. In the ROC graph, the gap between in-sample and out-of-sample ROC is also very significant. Altogether, it suggests that the model is overfitting and does not generalize well to unseen data.

From the confusion matrix, we can see that the model's recall is moderately low, at 0.5260. This means that the model only managed to correctly identify approximately half of the positive data points. However, the low FPR of 0.0712 means that its positive predictions are very reliable. Conversely, while the model managed to correctly identify most of the negative data points (specificity of 0.9288), the false negative rate of 0.3000 means that its negative predictions are far less reliable than its positive ones.

In [ ]:
model_dt = DecisionTreeClassifier().fit(X_train, y_train)
summary_dt = evaluate_model(model_dt)

### 6.4. Random RandomForestClassifier

In our random forest classifier, we can see that the accuracy remains high, but the disparity between in-sample and out-of-sample accuracy is almost as high as that of the decision tree classifier's. This disparity is also observed in the training and testing F1 scores, as well as the training and testing AUC values. However, the gap is much smaller than that of the decision tree model, which indicates that the degree of overfitting is smaller in our random forest model.

From the confusion matrix, we can see that the model has a moderately low recall of 0.5525. However, the  false positive rate of 0.0723 suggests that the model's positive predictions are very reliable. In comparison, the model managed to correctly identify a much larger portion of the negative data points but the false negative rate of 0.2929 is rather high, which means that its negative predictions are not as reliable as its positive predictions.

In [ ]:
model_rf = RandomForestClassifier().fit(X_train, y_train)
summary_rf = evaluate_model(model_rf)

### 6.5. XGBoost Clasifier

In our XGBoost model, training and testing accuracies remain high at 0.8604 and 0.8471 respectively. Despite having smaller training and testing f1 scores of 0.6760 and 0.6410 respectively, the difference in in and out-of-sample performance is much smaller than the other two tree-based algorithms, and this is further supported by the smaller gap between the training and testing ROC curves.

From the confusion matrix, we can see that the recall is relatively higher, at 0.5687. This means that the model correctly identified slightly over half of the positive cases (income > 50k group). However, it comes at a very low FPR of 0.0650, which means that the data points which the model identifies to be from the high income group are very likely to actually belong in that group.

In [ ]:
model_xgb = XGBClassifier().fit(X_train, y_train)
summary_xgb = evaluate_model(model_xgb)

### 6.6. Multi-layer Perceptron

For our neural network model, we used multi-layer perceptrons, which is a feed-forward architecture. The in-sample accuracy of 0.8646 is a bit larger than the out-of-sample accuracy of 0.8412. While the training f1 score of 0.6753 is very high and indicates greater ability to distinguish between the two income groups, the out-of-sample f1 score is much lower at 0.6123. Further, the gap between the training and testing ROC curves are also quite substantial. This suggests that the degree of overfitting is also relatively high, and the variance in out-of-sample performance may be very large. 

From the confusion matrix, we can see that the model has a relatively low recall of 0.5226 and low false positive and negative rates of 0.0583 and 0.2609 respectively. This means that the model is not as good at distinguishing data points from the two income groups from each other as the other models.

In [ ]:
model_mlp = MLPClassifier().fit(X_train, y_train)
summary_mlp = evaluate_model(model_mlp)

### 6.7. Naiive Bayes

Our Naiive Bayes model performed the worst. It has a training and testing accuracy of 0.6986 and 0.7010 respectively, which is much lower than the other models'. However, the disparity between training and testing f1 scores of 0.5918 and 0.5926 respectively indicates that the model performance is very consistent. 

From the confusion matrix, we can also see that the model has a very high sensitivity of 0.9137 - this means that the model is able to correctly identify a large portion of the positive data points (income > 50k group). However, it comes at the cost of a very high false positive rate of 0.3862. 

In [ ]:
model_nb = GaussianNB().fit(X_train, y_train)
summary_nb = evaluate_model(model_nb)

### 6.8 Support Vector Classifier

Our support vector classifier took the longest time to train, but the performance is relatively good and consistent. It has a high training and testing accuracy of 0.8497 and 0.8459 respectively, which are very close to each other. Similarly, its training and testing F1 scores of 0.6253 and 0.6071 also demonstrated moderate consistency between in and out-of-sample performance. From the ROC graph, we can see that the gap between the training and testing ROC are also very small, indicating that the model is able to distinguish between data points of the two income groups quite reliably, and also generalizes well to unseen data.

However, from its confusion matrix, we can see that its recall value is relatively low, at only 0.4962. This means that the model is comparatively worse at identifying the positive cases. However, its false positive rate of 0.0437 is also much lower, indicating that the model's positive predictions are much more reliable. Like the other models, our SVC model is also better at correctly identifying the negative data points, but with a lower degree of reliability.

In [ ]:
model_svc = SVC(probability=True).fit(X_train, y_train)
summary_svc = evaluate_model(model_svc)

In [ ]:
# Your models and metrics data
models = [
    "Logistic Regression", "Decision Tree",
    "Random Forest", "XGBoost",
    "MLP", "Naiive Bayes",
    "SVC"
]

# Summary of all models' performance (assuming these are dictionaries)
df_basic_metrics = [
    summary_lr["basic_metrics"],
    summary_dt["basic_metrics"],
    summary_rf["basic_metrics"],
    summary_xgb["basic_metrics"],
    summary_mlp["basic_metrics"],
    summary_nb["basic_metrics"],
    summary_svc["basic_metrics"]
]

df_cf_metrics = [
    summary_lr["cf_metrics"],
    summary_dt["cf_metrics"],
    summary_rf["cf_metrics"],
    summary_xgb["cf_metrics"],
    summary_mlp["cf_metrics"],
    summary_nb["cf_metrics"],
    summary_svc["cf_metrics"]
]

# Convert lists of dictionaries to DataFrames
df_basic_metrics = pd.DataFrame(df_basic_metrics, index=models)
df_cf_metrics = pd.DataFrame(df_cf_metrics, index=models)

# View basic metrics
df_basic_metrics

In [ ]:
# View cf metrics
df_cf_metrics